In [12]:
import pandas as pd
import numpy as np

import re
import holidays
from datetime import datetime, timedelta
from math import radians, sin, cos, sqrt, atan2

import warnings
from tqdm import tqdm

warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)

path = '/Users/Goo/Boaz/3차 미니프로젝트/Data/'

### 1. 지역 축제 정보
- 개최연도, 축제명, 예산(백만원), 방문객수, 개최 기간

In [2]:
# 지역 축제 정보
df2023 = pd.read_excel(path + '2023_지역축제 정보.xlsx')
df2022 = pd.read_excel(path + '2022_지역축제 정보.xlsx')
df2021 = pd.read_excel(path + '2021_지역축제 정보.xlsx')
df2019 = pd.read_excel(path + '2019_지역축제 정보.xlsx')
df2018 = pd.read_excel(path + '2018_지역축제 정보.xlsx')

In [3]:
# 데이터 클렌징
def data_cleansing(df):
    df["최초 개최년도"] = df["최초 개최년도"].astype(str).str[:4].apply(lambda x: np.nan if not x.isdigit() else x)
    df = df.dropna(subset=["최초 개최년도"])
    
    # 개최 기간 변수 생성
    df["개최 기간"] = df["개최연도"] - pd.to_numeric(df["최초 개최년도"])
    df.drop("최초 개최년도", axis=1, inplace=True)
    
    df["예산(백만원)"] = pd.to_numeric(df["예산(백만원)"], errors='coerce')
    df = df.dropna(subset=["예산(백만원)"])

    df["방문객수"] = pd.to_numeric(df["방문객수"], errors='coerce')
    df.dropna(subset=["방문객수"], inplace=True)
    df = df[df["방문객수"] != 0]
    df.reset_index(inplace=True, drop=True)
    
    return df

df2023 = data_cleansing(df2023)
df2022 = data_cleansing(df2022)
df2021 = data_cleansing(df2021)
df2019 = data_cleansing(df2019)
df2018 = data_cleansing(df2018)

In [4]:
# 단위 통일(2019년, 2018년 방문객수 단위가 천명)
df2019["방문객수"] *= 1000
df2018["방문객수"] *= 1000

In [5]:
df2023.head()

,개최연도,축제명,예산(백만원),방문객수,개최 기간
0,2023,서울드럼페스티벌,564.0,12500.0,24
1,2023,서울뮤직페스티벌,1062.0,12274.0,4
2,2023,서울국악축제,207.0,17442.0,4
3,2023,서울페스타 2023,2115.0,392000.0,1
4,2023,2023 한강 불빛 공연(드론 라이트 쇼),837.0,124360.0,0


In [6]:
df_festival = pd.concat([df2023, df2022, df2021, df2019, df2018], ignore_index=True)
df_festival

,개최연도,축제명,예산(백만원),방문객수,개최 기간
0,2023,서울드럼페스티벌,564.0,12500.0,24
1,2023,서울뮤직페스티벌,1062.0,12274.0,4
2,2023,서울국악축제,207.0,17442.0,4
3,2023,서울페스타 2023,2115.0,392000.0,1
4,2023,2023 한강 불빛 공연(드론 라이트 쇼),837.0,124360.0,0
...,...,...,...,...,...
3545,2018,거창크리스마스트리문화축제,112.0,40000.0,5
3546,2018,금원산얼음축제,100.0,15000.0,11
3547,2018,제23회 황매산철쭉제,190.0,510000.0,21
3548,2018,합천바캉스축제,248.0,57000.0,0


In [ ]:
# df_festival.to_csv(path + "지역 축제 정보.csv", index=False)

### 2. 전국 문화축제 표준데이터
- 축제명, 시도명, 시군구명, 행정동명, 위도, 경도, 서울과의 거리, 광역시와의 거리, 축제시작일, 축제종료일, 축제 일수, 주말/공휴일 유무, 개최 월, 개최 계절

In [45]:
df2 = pd.read_csv(path + '전국 문화축제 표준데이터.csv')
df2.head(3)

,ID,LCLAS_NM,MLSFC_NM,FCLTY_NM,CTPRVN_NM,SIGNGU_NM,LEGALDONG_CD,LEGALDONG_NM,ADSTRD_CD,ADSTRD_NM,RDNMADR_CD,RDNMADR_NM,ZIP_NO,GID_CD,FCLTY_LO,FCLTY_LA,OPMTN_PLACE_NM,FSTVL_BEGIN_DE,FSTVL_END_DE,FSTVL_CN,MNNST_NM,AUSPC_INSTT_NM,SUPRT_INSTT_NM,TEL_NO,HMPG_ADDR,RELATE_INFO_CN,데이터기준일자,PROVD_INSTT_CD,PROVD_INSTT_NM,LAST_CHG_DE,ORIGIN_NM,FILE_NM,BASE_DE
0,KC488PO22N000001,행사,행사,춘천연극제,강원도,춘천시,4211010700,옥천동,4.211058e+09,소양동,4.210000e+11,강원도 춘천시 서부대성로 71,24264.0,라사203869,127.731370,37.882374,봄내극장+석사천,2022-06-15,2022-10-29,코미디경연작+초청작+소소연극제+가족극+살롱연극+거리공연,(사)춘천연극제,(사)춘천연극제,강원도 춘천시청,033-241-4345,http://www.citf.or.kr/,NaN,2022-08-26,4180000,강원도 춘천시,20220826,문화체육관광부,KC_488_WNTY_CLTFSTVL_2022,20220826
1,KC488PO22N000002,행사,행사,제34회 춘천인형극제,강원도,춘천시,4211011900,사농동,4.211070e+09,신사우동,4.210000e+11,강원도 춘천시 영서로 3017,24235.0,라사193912,127.720105,37.921016,춘천인형극장+축제극장몸짓,2022-04-04,2022-12-25,국내외 인형극 공연 관람+경연대회+퍼레이드,(재)춘천인형극제,강원도 춘천시청+(재)춘천인형극제,문화체육관광부+강원도청,033-250-4391,http://www.cocobau.com/,NaN,2022-08-26,4180000,강원도 춘천시,20220826,문화체육관광부,KC_488_WNTY_CLTFSTVL_2022,20220826
2,KC488PO22N000003,행사,행사,인형극아카데미,강원도,춘천시,4211011900,사농동,4.211070e+09,신사우동,4.210000e+11,강원도 춘천시 영서로 3017,24235.0,라사193912,127.720105,37.921016,춘천인형극장,2022-03-01,2022-05-31,찾아가는 인형극 공연+국내외 초청작 인형극 공연+커튼콜 개최(온라인공연 인기작 3팀...,(재)춘천인형극제,강원도 춘천시청+(재)춘천인형극제,문화체육관광부+강원도청,033-250-3067,http://www.cocobau.com/,NaN,2022-08-26,4180000,강원도 춘천시,20220826,문화체육관광부,KC_488_WNTY_CLTFSTVL_2022,20220826


In [46]:
# 축제명, 시도명, 시군구명, 행정동명, 위도, 경도, 축제시작일, 축제종료일
df2.rename(columns={"FCLTY_NM":"축제명", "CTPRVN_NM":"시도명", "SIGNGU_NM":"시군구명", "ADSTRD_NM":"행정동명", "FCLTY_LA":"위도", "FCLTY_LO":"경도", "FSTVL_BEGIN_DE":"축제시작일", "FSTVL_END_DE":"축제종료일"}, inplace=True)

In [47]:
# 축제 일수 변수 생성
df2['축제시작일'] = pd.to_datetime(df2['축제시작일'])
df2['축제종료일'] = pd.to_datetime(df2['축제종료일'])

df2['축제 일수'] = (df2['축제종료일'] - df2['축제시작일']).dt.days + 1

In [48]:
# 개최 월 변수 생성
df2['개최 월'] = df2['축제시작일'].dt.month

In [49]:
# 개최 계절 변수 생성
season_map = {3: '봄', 4: '봄', 5: '봄', 6: '여름', 7: '여름', 8: '여름', 9: '가을', 10: '가을', 11: '가을', 12: '겨울', 1: '겨울', 2: '겨울'}
df2['개최 계절'] = df2['개최 월'].map(season_map)

In [50]:
# 위도, 경도를 활용한 거리 계산 함수(하버사인)
def haversine(lat1, lon1, lat2, lon2):
    # 지구 반지름 (km)
    R = 6371.0

    # 라디안으로 변환
    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    # 위도 및 경도의 차이 계산
    dlon = lon2 - lon1
    dlat = lat2 - lat1

    # Haversine 공식 적용
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    # 거리 계산
    distance = R * c

    return distance

In [51]:
# 서울과의 거리 변수 생성 (서울역과 지역 축제 위치 사이의 거리 계산)
# 서울역 위경도 (37.554623, 126.970609)
seoul_station = (37.554623, 126.970609)
df2['서울과의 거리'] = df2.apply(lambda row: haversine(seoul_station[0], seoul_station[1], row['위도'], row['경도']), axis=1)

In [52]:
# 서울을 제외한 가장 가까운 광역시와의 거리 변수 생성
# 한국의 광역시 위경도
cities = {
    'Busan': (35.1796, 129.0756),
    'Daegu': (35.8714, 128.6014),
    'Incheon': (37.4563, 126.7052),
    'Gwangju': (35.1595, 126.8526),
    'Daejeon': (36.3504, 127.3845),
    'Ulsan': (35.5384, 129.3114)
}

# 가장 가까운 광역시와의 거리 계산 함수
def Gwangyeok_city(lat, lon, cities):
    min_dist = float('inf')
    nearest_city = None
    for city, (city_lat, city_lon) in cities.items():
        dist = haversine(lat, lon, city_lat, city_lon)
        if dist < min_dist:
            min_dist = dist
            nearest_city = city
    return nearest_city, min_dist

df2['광역시와의 거리'] = df2.apply(lambda row: Gwangyeok_city(row['위도'], row['경도'], cities)[1], axis=1)

In [53]:
# 위, 경도가 없는 행 삭제
df2 = df2.dropna(subset=['위도', '경도'])

In [54]:
# 축제 기간 중 주말/공휴일 포함 유무 변수 생성
# 한국의 공휴일 설정
kr_holidays = holidays.KR()

def weekend_holiday(start_date, end_date):
    date_range = pd.date_range(start=start_date, end=end_date)
    for single_date in date_range:
        if single_date.weekday() >= 5 or single_date in kr_holidays:
            return 1
    return 0

df2['주말/공휴일 유무'] = df2.apply(lambda row: weekend_holiday(row['축제시작일'], row['축제종료일']), axis=1)

In [55]:
# 필요 변수 추출
df2 = df2[["축제명","시도명","시군구명","행정동명","위도","경도","서울과의 거리","광역시와의 거리","축제시작일","축제종료일","축제 일수","주말/공휴일 유무","개최 월","개최 계절"]]
df2.head(3)

,축제명,시도명,시군구명,행정동명,위도,경도,서울과의 거리,광역시와의 거리,축제시작일,축제종료일,축제 일수,주말/공휴일 유무,개최 월,개최 계절
0,춘천연극제,강원도,춘천시,소양동,37.882374,127.731370,76.195488,101.990624,2022-06-15,2022-10-29,137,1,6,여름
1,제34회 춘천인형극제,강원도,춘천시,신사우동,37.921016,127.720105,77.482290,103.176546,2022-04-04,2022-12-25,266,1,4,봄
2,인형극아카데미,강원도,춘천시,신사우동,37.921016,127.720105,77.482290,103.176546,2022-03-01,2022-05-31,92,1,3,봄


In [56]:
# df2.to_csv(path + "지역 축제 표준데이터.csv", index=False)

### 3. 행정동별 인구수
- 2018, 2019, 2021, 2022, 2023
- 시군구명, 행정동명, 연도, 10대 인구수, 20대 인구수, 30대 인구수, 40대 인구수, 50대 인구수, 60대 이상 인구수, 평균 나이, 고령 지수

In [236]:
df3=pd.read_csv(path + '2018_2023_행정동별_인구수.csv', encoding="CP949")
df3.head(3)

,행정구역,2018년_계_총인구수,2018년_계_연령구간인구수,2018년_계_0~9세,2018년_계_10~19세,2018년_계_20~29세,2018년_계_30~39세,2018년_계_40~49세,2018년_계_50~59세,2018년_계_60~69세,2018년_계_70~79세,2018년_계_80~89세,2018년_계_90~99세,2018년_계_100세 이상,2019년_계_총인구수,2019년_계_연령구간인구수,2019년_계_0~9세,2019년_계_10~19세,2019년_계_20~29세,2019년_계_30~39세,2019년_계_40~49세,2019년_계_50~59세,2019년_계_60~69세,2019년_계_70~79세,2019년_계_80~89세,2019년_계_90~99세,2019년_계_100세 이상,2020년_계_총인구수,2020년_계_연령구간인구수,2020년_계_0~9세,2020년_계_10~19세,2020년_계_20~29세,2020년_계_30~39세,2020년_계_40~49세,2020년_계_50~59세,2020년_계_60~69세,2020년_계_70~79세,2020년_계_80~89세,2020년_계_90~99세,2020년_계_100세 이상,2021년_계_총인구수,2021년_계_연령구간인구수,2021년_계_0~9세,2021년_계_10~19세,2021년_계_20~29세,2021년_계_30~39세,2021년_계_40~49세,2021년_계_50~59세,2021년_계_60~69세,2021년_계_70~79세,2021년_계_80~89세,2021년_계_90~99세,2021년_계_100세 이상,2022년_계_총인구수,2022년_계_연령구간인구수,2022년_계_0~9세,2022년_계_10~19세,2022년_계_20~29세,2022년_계_30~39세,2022년_계_40~49세,2022년_계_50~59세,2022년_계_60~69세,2022년_계_70~79세,2022년_계_80~89세,2022년_계_90~99세,2022년_계_100세 이상,2023년_계_총인구수,2023년_계_연령구간인구수,2023년_계_0~9세,2023년_계_10~19세,2023년_계_20~29세,2023년_계_30~39세,2023년_계_40~49세,2023년_계_50~59세,2023년_계_60~69세,2023년_계_70~79세,2023년_계_80~89세,2023년_계_90~99세,2023년_계_100세 이상
0,서울특별시 (1100000000),"9,765,623","9,765,623","690,368","855,121","1,449,790","1,548,770","1,590,374","1,546,954","1,142,415","660,825","238,869","36,294","5,843","9,729,107","9,729,107","662,693","819,571","1,456,623","1,509,959","1,560,983","1,539,968","1,189,802","684,087","259,487","39,690","6,244","9,668,465","9,668,465","623,745","781,866","1,460,411","1,475,106","1,528,182","1,523,556","1,243,954","704,076","278,094","42,683","6,792","9,509,458","9,509,458","583,683","756,136","1,422,277","1,439,279","1,480,780","1,507,194","1,285,922","695,800","295,009","41,735","1,643","9,428,372","9,428,372","545,317","749,118","1,389,483","1,425,808","1,449,706","1,492,802","1,297,787","713,496","321,209","42,354","1,292","9,386,034","9,386,034","513,562","733,211","1,355,811","1,427,405","1,420,073","1,495,557","1,315,938","738,469","339,214","45,346","1,448"
1,서울특별시 종로구 (1111000000),"153,065","153,065","8,588","12,383","23,765","21,231","23,849","25,744","18,581","12,517","5,216",960,231,"151,290","151,290","8,211","11,748","23,572","20,376","23,077","25,714","19,128","12,538","5,673","1,000",253,"149,384","149,384","7,527","11,013","23,416","19,943","22,231","25,459","19,948","12,500","5,978","1,093",276,"144,683","144,683","6,984","10,381","22,802","19,470","20,969","24,957","20,352","11,637","6,149",950,32,"141,379","141,379","6,403","10,087","22,284","19,342","19,944","24,216","20,301","11,404","6,454",915,29,"139,417","139,417","5,919","9,739","21,496","19,322","19,194","23,915","20,689","11,503","6,644",961,35
2,서울특별시 종로구 청운효자동(1111051500),"13,045","13,045",960,"1,344","1,653","1,899","2,312","1,987","1,306","1,039",454,75,16,"12,734","12,734",901,"1,258","1,628","1,760","2,251","1,986","1,334","1,025",497,77,17,"12,431","12,431",827,"1,165","1,596","1,694","2,151","1,976","1,388","1,010",529,75,20,"11,978","11,978",738,"1,148","1,566","1,581","2,075","1,925","1,407",932,529,75,2,"11,628","11,628",679,"1,107","1,449","1,564","2,018","1,888","1,397",915,539,69,3,"11,349","11,349",596,"1,084","1,402","1,512","1,891","1,914","1,414",901,552,80,3


In [237]:
# 행정구역 제외 타입 변경
cols = df3.columns.difference(['행정구역'])
df3[cols] = df3[cols].replace(',', '', regex=True).fillna(0).astype(int)

In [238]:
# 행정동 별로 남기기 위해서, 동이 없는 데이터 제거
def process_address(text):
    
    text = re.sub(r'\(\d+\)', '', text)
    text = text.strip()
    lines = text.split('\n')
    filtered_lines = [line for line in lines if line.endswith('동')]
    
    return '\n'.join(filtered_lines)

df3["행정구역"] = df3["행정구역"].apply(process_address)
df3 = df3[df3["행정구역"] != '']

In [239]:
# 시군구명, 행정동명 분리
def split_address(address):

    parts = address.split()
    시군구명 = parts[-2]
    행정동명 = parts[-1]
    return 시군구명, 행정동명

df3[['시군구명', '행정동명']] = df3['행정구역'].apply(lambda x: pd.Series(split_address(x)))

In [240]:
# 변수 전처리
df3.rename(columns = {
"2018년_계_0~9세":"2018 10대미만_인구수",
"2019년_계_0~9세":"2019 10대미만_인구수",   
"2021년_계_0~9세":"2021 10대미만_인구수",
"2022년_계_0~9세":"2022 10대미만_인구수",
"2023년_계_0~9세":"2023 10대미만_인구수",    
    
"2018년_계_10~19세":"2018 10대_인구수",
"2019년_계_10~19세":"2019 10대_인구수",   
"2021년_계_10~19세":"2021 10대_인구수",
"2022년_계_10~19세":"2022 10대_인구수",
"2023년_계_10~19세":"2023 10대_인구수",

"2018년_계_20~29세":"2018 20대_인구수",
"2019년_계_20~29세":"2019 20대_인구수",
"2021년_계_20~29세":"2021 20대_인구수",
"2022년_계_20~29세":"2022 20대_인구수",
"2023년_계_20~29세":"2023 20대_인구수",
    
"2018년_계_30~39세":"2018 30대_인구수",
"2019년_계_30~39세":"2019 30대_인구수",
"2021년_계_30~39세":"2021 30대_인구수",
"2022년_계_30~39세":"2022 30대_인구수",
"2023년_계_30~39세":"2023 30대_인구수",
    
"2018년_계_40~49세":"2018 40대_인구수",
"2019년_계_40~49세":"2019 40대_인구수",
"2021년_계_40~49세":"2021 40대_인구수",
"2022년_계_40~49세":"2022 40대_인구수",
"2023년_계_40~49세":"2023 40대_인구수",
    
"2018년_계_50~59세":"2018 50대_인구수",
"2019년_계_50~59세":"2019 50대_인구수",
"2021년_계_50~59세":"2021 50대_인구수",
"2022년_계_50~59세":"2022 50대_인구수",
"2023년_계_50~59세":"2023 50대_인구수",

"2018년_계_60~69세":"2018 60대_인구수",
"2019년_계_60~69세":"2019 60대_인구수",
"2021년_계_60~69세":"2021 60대_인구수",
"2022년_계_60~69세":"2022 60대_인구수",
"2023년_계_60~69세":"2023 60대_인구수",

"2018년_계_70~79세":"2018 70대_인구수",
"2019년_계_70~79세":"2019 70대_인구수",
"2021년_계_70~79세":"2021 70대_인구수",
"2022년_계_70~79세":"2022 70대_인구수",
"2023년_계_70~79세":"2023 70대_인구수",

"2018년_계_80~89세":"2018 80대_인구수",
"2019년_계_80~89세":"2019 80대_인구수",
"2021년_계_80~89세":"2021 80대_인구수",
"2022년_계_80~89세":"2022 80대_인구수",
"2023년_계_80~89세":"2023 80대_인구수",

"2018년_계_90~99세":"2018 90대_인구수",
"2019년_계_90~99세":"2019 90대_인구수",
"2021년_계_90~99세":"2021 90대_인구수",
"2022년_계_90~99세":"2022 90대_인구수",
"2023년_계_90~99세":"2023 90대_인구수",
    
"2018년_계_100세 이상":"2018 100대_인구수",
"2019년_계_100세 이상":"2019 100대_인구수",
"2021년_계_100세 이상":"2021 100대_인구수",
"2022년_계_100세 이상":"2022 100대_인구수",
"2023년_계_100세 이상":"2023 100대_인구수"}, inplace = True)

In [241]:
# 필요 변수 추출
df3 = df3[["시군구명", "행정동명",
           "2018 10대미만_인구수","2018 10대_인구수","2018 20대_인구수","2018 30대_인구수","2018 40대_인구수","2018 50대_인구수","2018 60대_인구수","2018 70대_인구수","2018 80대_인구수","2018 90대_인구수","2018 100대_인구수",
           "2019 10대미만_인구수","2019 10대_인구수","2019 20대_인구수","2019 30대_인구수","2019 40대_인구수","2019 50대_인구수","2019 60대_인구수","2019 70대_인구수","2019 80대_인구수","2019 90대_인구수","2019 100대_인구수",
           "2021 10대미만_인구수","2021 10대_인구수","2021 20대_인구수","2021 30대_인구수","2021 40대_인구수","2021 50대_인구수","2021 60대_인구수","2021 70대_인구수","2021 80대_인구수","2021 90대_인구수","2021 100대_인구수",
           "2022 10대미만_인구수","2022 10대_인구수","2022 20대_인구수","2022 30대_인구수","2022 40대_인구수","2022 50대_인구수","2022 60대_인구수","2022 70대_인구수","2022 80대_인구수","2022 90대_인구수","2022 100대_인구수",
           "2023 10대미만_인구수","2023 10대_인구수","2023 20대_인구수","2023 30대_인구수","2023 40대_인구수","2023 50대_인구수","2023 60대_인구수","2023 70대_인구수","2023 80대_인구수","2023 90대_인구수","2023 100대_인구수"]]

In [242]:
# 연도와 인구수로 칼럼 분리 및 공통 칼럼으로 병합
plp = pd.melt(df3, id_vars=["시군구명", "행정동명"], var_name="연도_연령대", value_name="인구수")
plp[['연도', '연령대']] = plp['연도_연령대'].str.split(' ', expand=True)
plp.drop(columns=['연도_연령대'], inplace=True)

plp['연령대'] = plp['연령대'].apply(lambda x: x.split('_')[0])
plp = plp.pivot_table(index=["시군구명", "행정동명", "연도"], columns="연령대", values="인구수").reset_index()

plp.columns.name = None
plp.columns = ['시군구명', '행정동명', '연도', '10대미만_인구수', '10대_인구수', '20대_인구수', '30대_인구수', '40대_인구수', '50대_인구수', '60대_인구수', '70대_인구수', '80대_인구수', '90대_인구수', '100대_인구수']
plp.head(10)

,시군구명,행정동명,연도,10대미만_인구수,10대_인구수,20대_인구수,30대_인구수,40대_인구수,50대_인구수,60대_인구수,70대_인구수,80대_인구수,90대_인구수,100대_인구수
0,강남구,개포1동,2018,9.0,729.0,427.0,1024.0,858.0,1107.0,1359.0,1307.0,689.0,203.0,59.0
1,강남구,개포1동,2019,10.0,620.0,385.0,850.0,733.0,971.0,1168.0,1207.0,687.0,209.0,55.0
2,강남구,개포1동,2021,5.0,605.0,345.0,704.0,632.0,919.0,1023.0,1219.0,719.0,248.0,51.0
3,강남구,개포1동,2022,2.0,696.0,426.0,701.0,686.0,1023.0,1095.0,1191.0,764.0,274.0,43.0
4,강남구,개포1동,2023,2.0,1380.0,848.0,1180.0,1186.0,2066.0,1888.0,1865.0,1081.0,344.0,46.0
5,강남구,개포2동,2018,8.0,1816.0,793.0,2019.0,1484.0,2236.0,2627.0,1811.0,740.0,244.0,51.0
6,강남구,개포2동,2019,9.0,2638.0,1619.0,2966.0,2892.0,3636.0,3837.0,2825.0,1176.0,331.0,72.0
7,강남구,개포2동,2021,1.0,3505.0,2346.0,3531.0,3234.0,4870.0,4831.0,3472.0,1559.0,437.0,88.0
8,강남구,개포2동,2022,2.0,3685.0,2375.0,3521.0,3511.0,5026.0,4988.0,3498.0,1791.0,459.0,91.0
9,강남구,개포2동,2023,3.0,5139.0,3631.0,4684.0,6165.0,7508.0,6677.0,4699.0,2450.0,621.0,107.0


In [243]:
# 각 나이대의 중앙값
mid_age = {
    '10대미만_인구수': 5,
    '10대_인구수': 15,
    '20대_인구수': 25,
    '30대_인구수': 35,
    '40대_인구수': 45,
    '50대_인구수': 55,
    '60대_인구수': 65,
    '70대_인구수': 75,
    '80대_인구수': 85,
    '90대_인구수': 95,
    '100대_인구수': 100
}

In [244]:
# 총인구수 계산
plp['총인구수'] = plp[["10대미만_인구수","10대_인구수","20대_인구수","30대_인구수","40대_인구수","50대_인구수","60대_인구수","70대_인구수","80대_인구수","90대_인구수","100대_인구수"]].sum(axis=1)

# 평균 나이 변수 생성
plp['평균 나이'] = (plp['10대미만_인구수'] * mid_age['10대미만_인구수'] +
                   plp['10대_인구수'] * mid_age['10대_인구수'] +
                   plp['20대_인구수'] * mid_age['20대_인구수'] +
                   plp['30대_인구수'] * mid_age['30대_인구수'] +
                   plp['40대_인구수'] * mid_age['40대_인구수'] +
                   plp['50대_인구수'] * mid_age['50대_인구수'] +
                   plp['60대_인구수'] * mid_age['60대_인구수'] +
                   plp['70대_인구수'] * mid_age['70대_인구수'] +
                   plp['80대_인구수'] * mid_age['80대_인구수'] +
                   plp['90대_인구수'] * mid_age['90대_인구수'] +
                   plp['100대_인구수'] * mid_age['100대_인구수']) / plp['총인구수']

In [245]:
# 60대 이상 인구수 변수 생성
plp['60대이상_인구수'] = (plp['60대_인구수'] + plp['70대_인구수'] + plp['80대_인구수'] + plp['90대_인구수'] + plp['100대_인구수'])

# 고령 지수 변수 생성
plp['고령 지수'] = plp['60대이상_인구수'] / plp['총인구수']

In [246]:
plp.drop(["10대미만_인구수","60대_인구수","70대_인구수", "80대_인구수", "90대_인구수", "100대_인구수", "총인구수"],axis=1, inplace=True)
plp.head(3)

,시군구명,행정동명,연도,10대_인구수,20대_인구수,30대_인구수,40대_인구수,50대_인구수,평균 나이,60대이상_인구수,고령 지수
0,강남구,개포1동,2018,729.0,427.0,1024.0,858.0,1107.0,54.960752,3617.0,0.465448
1,강남구,개포1동,2019,620.0,385.0,850.0,733.0,971.0,55.882524,3326.0,0.482379
2,강남구,개포1동,2021,605.0,345.0,704.0,632.0,919.0,57.039413,3260.0,0.503864


In [247]:
# plp.to_csv(path + "행정동별 인구수.csv", index=False)

### 4. 문화축제 검색량 데이터
- 축제명, 검색량, 연도, 월

In [23]:
search = pd.DataFrame()

# 2020년 01월 ~ 2023년 12월
for year in range(2020, 2024):
    for month in range(1, 13):
        # 문자열 포매팅을 활용해 파일 로드
        filename = f"/Users/Goo/Boaz/3차 미니프로젝트/Data/검색량/DM_CLTUR_FSTVL_SCCNT_RESULT_{year}{month:02d}.csv"
        
        df = pd.read_csv(filename)
        
        df = df[["SCCNT_YM", "SRCHWRD_NM", "SCCNT_VALUE"]]
        df.rename(columns={"SCCNT_YM": "검색량년월", "SRCHWRD_NM": "축제명", "SCCNT_VALUE": "검색량"}, inplace=True)
        
        # 연도와 월로 추출
        df['연도'] = df["검색량년월"].apply(lambda x: x.split('-')[0])
        df['월'] = df["검색량년월"].apply(lambda x: x.split('-')[1])
        df.drop("검색량년월", axis=1, inplace=True)
        
        # 48개 데이터셋 병합
        search = pd.concat([search, df])

search.loc[search["연도"] == "2020", "연도"] = "2019"
search["월"]=search["월"].astype('int')
search.drop_duplicates(inplace=True)
search.reset_index(inplace=True, drop=True)

In [24]:
# 축제명, 월을 기준으로 2018년도 축제 검색량 데이터 생성
# 19, 21, 22, 23년도 데이터의 축제별 월별 평균을 사용
search_mean = search.groupby(['축제명', '월'])['검색량'].mean().reset_index()

# 19~23의 고유한 모든 축제를 사용
# 병합시 존재하지 않는 축제는 포함되지 않을 것이기에 상관없음
unique_festival = search["축제명"].unique()

row_2018 = []
for festival in unique_festival:
    for month in range(1, 13):
        festival_mean = search_mean[(search_mean['축제명'] == festival) & (search_mean['월'] == month)]['검색량'].mean()
        row_2018.append({'축제명': festival, '월': month, '검색량': festival_mean, '연도': 2018})

df_2018 = pd.DataFrame(row_2018)
df_2018

,축제명,월,검색량,연도
0,부산바다축제,1,202.5,2018
1,부산바다축제,2,215.0,2018
2,부산바다축제,3,352.5,2018
3,부산바다축제,4,960.0,2018
4,부산바다축제,5,2085.0,2018
...,...,...,...,...
9535,도리사해맞이행사,8,NaN,2018
9536,도리사해맞이행사,9,NaN,2018
9537,도리사해맞이행사,10,NaN,2018
9538,도리사해맞이행사,11,NaN,2018


In [25]:
# 결측치 제거 및 병합
df_2018.dropna(inplace=True)
search = pd.concat([df_2018, search])
search.reset_index(inplace=True, drop=True)
search

,축제명,월,검색량,연도
0,부산바다축제,1,202.5,2018
1,부산바다축제,2,215.0,2018
2,부산바다축제,3,352.5,2018
3,부산바다축제,4,960.0,2018
4,부산바다축제,5,2085.0,2018
...,...,...,...,...
30606,화순국화향연,12,20.0,2023
30607,황태축제,12,22.0,2023
30608,횡성한우축제,12,2390.0,2023
30609,효문화뿌리축제,12,11.0,2023


In [ ]:
# search.to_csv(path + "문화축제 검색량.csv", index=False)

### 5. 기상청 종관기상관측(ASOS) 데이터
- 일시, 평균 기온, 일강수량, 평균 풍속, 평균 상대습도, 위도, 경도

In [26]:
# 기상청 종관기상관측(ASOS) 데이터
file = [
    "기상청 날씨 180105.csv",
    "기상청 날씨 190105.csv",
    "기상청 날씨 210105.csv",
    "기상청 날씨 220105.csv",
    "기상청 날씨 230105.csv",
    "기상청 날씨 180612.csv",
    "기상청 날씨 190612.csv",
    "기상청 날씨 210612.csv",
    "기상청 날씨 220612.csv",
    "기상청 날씨 230612.csv"
]

df5_1 = []
for file_name in file:
    df = pd.read_csv(path + "날씨/" + file_name, encoding="CP949")
    df5_1.append(df)

df5_1 = pd.concat(df5_1, ignore_index=True)
df5_1

,지점,지점명,일시,평균기온(°C),일강수량(mm),평균 풍속(m/s),평균 상대습도(%)
0,90,속초,2018-01-01,1.0,NaN,2.6,21.3
1,90,속초,2018-01-02,1.5,NaN,2.9,21.8
2,90,속초,2018-01-03,-1.6,NaN,1.6,29.9
3,90,속초,2018-01-04,-1.0,NaN,1.5,53.3
4,90,속초,2018-01-05,1.5,NaN,1.2,45.4
...,...,...,...,...,...,...,...
173971,296,북부산,2023-12-27,4.4,NaN,1.1,62.4
173972,296,북부산,2023-12-28,3.8,NaN,0.9,66.6
173973,296,북부산,2023-12-29,2.5,NaN,0.8,66.1
173974,296,북부산,2023-12-30,4.2,1.0,0.7,73.8


In [27]:
# 비의 유무 파악
df5_1['일강수량(mm)'] = df5_1['일강수량(mm)'].fillna(0)
df5_1['비 유무'] = df5_1['일강수량(mm)'].apply(lambda x: 0 if x == 0 else 1)
df5_1.drop('일강수량(mm)', axis=1, inplace=True)

In [28]:
df5_1.head()

,지점,지점명,일시,평균기온(°C),평균 풍속(m/s),평균 상대습도(%),비 유무
0,90,속초,2018-01-01,1.0,2.6,21.3,0
1,90,속초,2018-01-02,1.5,2.9,21.8,0
2,90,속초,2018-01-03,-1.6,1.6,29.9,0
3,90,속초,2018-01-04,-1.0,1.5,53.3,0
4,90,속초,2018-01-05,1.5,1.2,45.4,0


In [29]:
# 관측 지점 위, 경도 데이터
df5_2 = pd.read_csv(path + '날씨/기상청 관측 지점 정보.csv', encoding="CP949")
df5_2 = df5_2[["지점", "지점명", "위도", "경도"]]
df5_2 = df5_2.drop_duplicates()
df5_2.head()

,지점,지점명,위도,경도
0,3,선봉,42.3167,130.4000
1,5,삼지연,41.8167,128.3167
2,8,청진,41.7833,129.8167
3,12,안면도(감),36.5333,126.3167
4,13,고산센터,33.3000,126.1600


In [30]:
# 데이터 병합
df5 = pd.merge(df5_1, df5_2[["지점", "위도", "경도"]], on='지점', how='inner')
df5 = df5.dropna()
df5 = df5.drop_duplicates(subset=['지점', '일시'])
df5.drop(["지점", "지점명"], axis=1, inplace=True)
df5.reset_index(drop=True, inplace=True)

In [31]:
df5 

,일시,평균기온(°C),평균 풍속(m/s),평균 상대습도(%),비 유무,위도,경도
0,2018-01-01,1.0,2.6,21.3,0,38.2509,128.5647
1,2018-01-02,1.5,2.9,21.8,0,38.2509,128.5647
2,2018-01-03,-1.6,1.6,29.9,0,38.2509,128.5647
3,2018-01-04,-1.0,1.5,53.3,0,38.2509,128.5647
4,2018-01-05,1.5,1.2,45.4,0,38.2509,128.5647
...,...,...,...,...,...,...,...
173426,2023-12-27,4.4,1.1,62.4,0,35.2178,128.9602
173427,2023-12-28,3.8,0.9,66.6,0,35.2178,128.9602
173428,2023-12-29,2.5,0.8,66.1,0,35.2178,128.9602
173429,2023-12-30,4.2,0.7,73.8,1,35.2178,128.9602


In [32]:
# df5.to_csv(path + "지역 축제 날씨.csv", index=False)

### 6. 한국철도공사 역 위치 정보 데이터
- 위도, 경도

In [33]:
# 한국철도공사 역 위, 경도 데이터
df6 = pd.read_csv(path + '한국철도공사 역 위치 정보.csv', encoding="CP949")
df6 = df6[["위도", "경도"]]
df6

,위도,경도
0,37.554730,126.970800
1,37.529910,126.964800
2,37.580600,126.895900
3,37.364500,126.495200
4,37.898770,126.709800
...,...,...
197,35.050000,127.480000
198,35.060000,127.760000
199,35.164750,129.060072
200,35.538514,129.353265


In [ ]:
# df6.to_csv(path + "KTX 위경도.csv", index=False)

### 7. 전국 시군구 단위 소득 구간대별 주민 비율
- 위도, 경도

In [294]:
# 전국 시군구 단위 소득 구간대별 주민 비율 데이터
df7 = pd.read_csv(path + '전국 시군구 단위 소득 구간대별 주민 비율.csv')
df7.head(3)

,BASE_YM,SIGNGU_NM,SIGNGU_CD,INCOME_TWO_TMW_INHBT_RATE,INCOME_THREE_TMW_INHBT_RATE,INCOME_FOUR_TMW_INHBT_RATE,INCOME_FIVE_TMW_INHBT_RATE,INCOME_SIX_TMW_INHBT_RATE,INCOME_SEVEN_TMW_INHBT_RATE,INCOME_SEVEN_TMW_ABOVE_INHBT_RATE
0,202403,강원특별자치도 강릉시,51150,0.14,0.37,0.25,0.11,0.05,0.03,0.04
1,202403,강원특별자치도 고성군,51820,0.18,0.42,0.24,0.08,0.04,0.02,0.03
2,202403,강원특별자치도 동해시,51170,0.14,0.36,0.25,0.11,0.06,0.03,0.05


In [295]:
df7.rename(columns = {
"SIGNGU_NM":"시군구명",
"INCOME_TWO_TMW_INHBT_RATE":"소득2천만원주민비율",   
"INCOME_THREE_TMW_INHBT_RATE":"소득3천만원주민비율",
"INCOME_FOUR_TMW_INHBT_RATE":"소득4천만원주민비율",
"INCOME_FIVE_TMW_INHBT_RATE":"소득5천만원주민비율",    
"INCOME_SIX_TMW_INHBT_RATE":"소득6천만원주민비율",
"INCOME_SEVEN_TMW_INHBT_RATE":"소득7천만원주민비율",   
"INCOME_SEVEN_TMW_ABOVE_INHBT_RATE":"소득7천만원이상주민비율"}, inplace = True)

In [296]:
# 시군구명 분리 함수
# 도 + 시 or 도 + 시 + 구, or 도 + 시 + 군의 구조로 이루어져있음.
# 시군구명 기준으로 병합을 진행해야하기에, 데이터 손실을 줄이고자 도를 제외한 데이터를 시군구명_1, 시군구명_2에 담는다
# 공백이 한개인 경우, 시군구명_1, 시군구명_2에 중복으로 담고
# 공백이 두개인 경우, 따로 담는다.

def sigungoo(row):
    sigungoo1 = ''
    sigungoo2 = ''
    parts = row['시군구명'].split(' ')
    if len(parts) == 2:
        sigungoo1 = parts[1]
        sigungoo2 = parts[1]
    elif len(parts) == 3:
        sigungoo1 = parts[1]
        sigungoo2 = parts[2]
    return pd.Series([sigungoo1, sigungoo2])

df7[['시군구명_1', '시군구명_2']] = df7.apply(sigungoo, axis=1)

In [297]:
df7.drop(["BASE_YM", "시군구명", "SIGNGU_CD"], axis=1, inplace=True)

In [298]:
# 시군구명_1을 기준으로 그룹화하여 각 그룹에 대해 소득 변수들의 평균을 계산
df7 = df7.groupby('시군구명_1').agg({
    '소득2천만원주민비율': 'mean',
    '소득3천만원주민비율': 'mean',
    '소득4천만원주민비율': 'mean',
    '소득5천만원주민비율': 'mean',
    '소득6천만원주민비율': 'mean',
    '소득7천만원주민비율': 'mean',
    '소득7천만원이상주민비율': 'mean'
})

df7.reset_index(inplace=True)
df7.head()

,시군구명_1,소득2천만원주민비율,소득3천만원주민비율,소득4천만원주민비율,소득5천만원주민비율,소득6천만원주민비율,소득7천만원주민비율,소득7천만원이상주민비율
0,,0.10,0.26,0.25,0.13,0.08,0.06,0.12
1,가평군,0.14,0.40,0.25,0.10,0.05,0.02,0.04
2,강남구,0.08,0.28,0.21,0.13,0.07,0.05,0.18
3,강동구,0.09,0.34,0.26,0.13,0.06,0.04,0.08
4,강릉시,0.14,0.37,0.25,0.11,0.05,0.03,0.04


In [299]:
# df7.to_csv(path + "소득 구간대별 주민 비율.csv", index=False)

----------------------------------------------------------------

## 데이터 병합

### 데이터 병합 순서

1. 문화축제를 기준으로 했기에, df4에 있는 축제명만 df1,df2에 남기고, 축제명 통일
2. 10km 이내의 KTX역 유무 파생변수 생성 (df2, df6)
3. df1, df2, df4, df6 병합
4. 시군구명, 행정동명, 연도 기준으로 df3 병합
5. 시군구명 기준 df7 병합
6. 축제 시작일 기준 df5 병합

In [372]:
df1 = pd.read_csv(path + '지역 축제 정보.csv')
df2 = pd.read_csv(path + '지역 축제 표준데이터.csv')
df3 = pd.read_csv(path + '행정동별 인구수.csv')
df4 = pd.read_csv(path + '문화축제 검색량.csv')
df5 = pd.read_csv(path + '지역 축제 날씨.csv')
df6 = pd.read_csv(path + 'KTX 위경도.csv')
df7 = pd.read_csv(path + "소득 구간대별 주민 비율.csv")

In [373]:
df1.head(2)

,개최연도,축제명,예산(백만원),방문객수,개최 기간
0,2023,서울드럼페스티벌,564.0,12500.0,24
1,2023,서울뮤직페스티벌,1062.0,12274.0,4


In [374]:
df2.head(2)

,축제명,시도명,시군구명,행정동명,위도,경도,서울과의 거리,광역시와의 거리,축제시작일,축제종료일,축제 일수,주말/공휴일 유무,개최 월,개최 계절
0,춘천연극제,강원도,춘천시,소양동,37.882374,127.731370,76.195488,101.990624,2022-06-15,2022-10-29,137,1,6,여름
1,제34회 춘천인형극제,강원도,춘천시,신사우동,37.921016,127.720105,77.482290,103.176546,2022-04-04,2022-12-25,266,1,4,봄


In [375]:
df3.head(2)

,시군구명,행정동명,연도,10대_인구수,20대_인구수,30대_인구수,40대_인구수,50대_인구수,평균 나이,60대이상_인구수,고령 지수
0,강남구,개포1동,2018,729.0,427.0,1024.0,858.0,1107.0,54.960752,3617.0,0.465448
1,강남구,개포1동,2019,620.0,385.0,850.0,733.0,971.0,55.882524,3326.0,0.482379


In [376]:
df4.head(2)

,축제명,월,검색량,연도
0,부산바다축제,1,202.5,2018
1,부산바다축제,2,215.0,2018


In [377]:
df5.head(2)

,일시,평균기온(°C),평균 풍속(m/s),평균 상대습도(%),비 유무,위도,경도
0,2018-01-01,1.0,2.6,21.3,0,38.2509,128.5647
1,2018-01-02,1.5,2.9,21.8,0,38.2509,128.5647


In [378]:
df6.head(2)

,위도,경도
0,37.55473,126.9708
1,37.52991,126.9648


In [379]:
df7.head(2)

,시군구명_1,소득2천만원주민비율,소득3천만원주민비율,소득4천만원주민비율,소득5천만원주민비율,소득6천만원주민비율,소득7천만원주민비율,소득7천만원이상주민비율
0,NaN,0.10,0.26,0.25,0.13,0.08,0.06,0.12
1,가평군,0.14,0.40,0.25,0.10,0.05,0.02,0.04


#### 1) 문화 축제를 기준으로 했기에, df4에 있는 축제명만 df1, df2에 남기고, 축제명 통일

In [380]:
# 포함 유무를 파악하기 위해 공백 제거
df1['공백제거'] = df1['축제명'].str.replace(' ', '')
df2['공백제거'] = df2['축제명'].str.replace(' ', '')
df4['공백제거'] = df4['축제명'].str.replace(' ', '')

In [381]:
# df4의 축제명을 기준으로 축제명 통일
# df4의 축제명을 df1이 포함하면 대체, 포함하지 않으면 제거
df4_festival_names = df4['공백제거'].tolist()

def replace_festival_name(row):
    for festival in df4_festival_names:
        if festival in row['공백제거']:
            return df4[df4['공백제거'] == festival]['축제명'].values[0]
    return None

df1['축제명'] = df1.apply(replace_festival_name, axis=1)
df1 = df1.dropna(subset=['축제명'])
df1 = df1.drop(columns=['공백제거'])
df1.reset_index(drop=True, inplace=True)

In [382]:
# df4의 축제명을 기준으로 축제명 통일
# df4의 축제명을 df2이 포함하면 대체, 포함하지 않으면 제거
df4_festival_names = df4['공백제거'].tolist()

def replace_festival_name(row):
    for festival in df4_festival_names:
        if festival in row['공백제거']:
            return df4[df4['공백제거'] == festival]['축제명'].values[0]
    return None

df2['축제명'] = df2.apply(replace_festival_name, axis=1)
df2 = df2.dropna(subset=['축제명'])
df2 = df2.drop(columns=['공백제거'])
df4 = df4.drop(columns=['공백제거'])
df2.reset_index(drop=True, inplace=True)

#### 2) 5km 이내의 KTX역 유무 파생변수 생성 (df2, df6)

In [383]:
# 위도, 경도를 활용한 거리 계산 함수(하버사인)
def haversine(lat1, lon1, lat2, lon2):
    # 지구 반지름 (km)
    R = 6371.0

    # 라디안으로 변환
    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    # 위도 및 경도의 차이 계산
    dlon = lon2 - lon1
    dlat = lat2 - lat1

    # Haversine 공식 적용
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    # 거리 계산
    distance = R * c

    return distance

In [384]:
# KTX역 유무 변수 생성
df2['KTX역 유무'] = 0

# 지역 축제 개최 주소로부터 5km 이내의 KTX역이 있는지 확인
# 있다: 1, 없다: 0
for i, row2 in df2.iterrows():
    for j, row6 in df6.iterrows():
        distance = haversine(row2['위도'], row2['경도'], row6['위도'], row6['경도'])
        if distance < 5:
            df2.at[i, 'KTX역 유무'] = 1
            break
            
df2 = df2.dropna(subset=['시도명', '시군구명', '행정동명'])
df2.head(3)

,축제명,시도명,시군구명,행정동명,위도,경도,서울과의 거리,광역시와의 거리,축제시작일,축제종료일,축제 일수,주말/공휴일 유무,개최 월,개최 계절,KTX역 유무
0,춘천연극제,강원도,춘천시,소양동,37.882374,127.731370,76.195488,101.990624,2022-06-15,2022-10-29,137,1,6,여름,0
1,춘천인형극제,강원도,춘천시,신사우동,37.921016,127.720105,77.482290,103.176546,2022-04-04,2022-12-25,266,1,4,봄,0
2,춘천마임축제,강원도,춘천시,효자1동,37.873097,127.727838,75.436768,101.244094,2022-05-22,2022-05-29,8,1,5,봄,0


#### 3) df1, df2, df4, df6 병합

In [385]:
# df1, df2, df6 병합
df_final = pd.merge(df1, df2, on='축제명', how='inner')
df_final = df_final.drop_duplicates(subset=['개최연도', '축제명'])
df_final['축제시작일'] = df_final['개최연도'].astype(str) + df_final['축제시작일'].str[4:]
df_final['축제종료일'] = df_final['개최연도'].astype(str) + df_final['축제종료일'].str[4:]

In [386]:
# 동일 기간 내 다른 축제 수 변수 생성
df_final['축제시작일'] = pd.to_datetime(df_final['축제시작일'])
df_final['축제종료일'] = pd.to_datetime(df_final['축제종료일'])

def same_time_festivals(row, df):
    start_date = row['축제시작일']
    end_date = row['축제종료일']
    count = df.apply(lambda x: x['축제시작일'] <= end_date and x['축제종료일'] >= start_date, axis=1).sum() - 1
    return count

df_final['동일 기간 축제 수'] = df_final.apply(same_time_festivals, df=df_final, axis=1)

In [387]:
# df4 병합
# 축제 시작일 기준 최근 3개월간 검색량 총합 변수 생성
df_final['연도'] = pd.to_datetime(df_final['축제시작일']).dt.year
df_final['월'] = pd.to_datetime(df_final['축제시작일']).dt.month

df_final = pd.merge(df_final, df4, on=['축제명', '연도', '월'], how='left')
df_final['검색량(3개월)'] = df_final.groupby('축제명')['검색량'].rolling(3, min_periods=1).sum().reset_index(drop=True)

# 결측치 보간(시도명 평균)
sido_mean = df_final.groupby('시도명')['검색량(3개월)'].mean()
df_final['검색량(3개월)'] = df_final.apply(lambda row: sido_mean[row['시도명']] if pd.isnull(row['검색량(3개월)']) else row['검색량(3개월)'], axis=1)

df_final.head(3)

,개최연도,축제명,예산(백만원),방문객수,개최 기간,시도명,시군구명,행정동명,위도,경도,서울과의 거리,광역시와의 거리,축제시작일,축제종료일,축제 일수,주말/공휴일 유무,개최 월,개최 계절,KTX역 유무,동일 기간 축제 수,연도,월,검색량,검색량(3개월)
0,2023,강남페스티벌,2532.0,56894.0,11,서울특별시,강남구,삼성1동,37.512561,127.058777,9.072810,31.818953,2023-09-30,2023-10-08,9,1,9,가을,0,49,2023,9,12640.0,47900.0
1,2019,강남페스티벌,2233.0,312502000.0,7,서울특별시,강남구,삼성1동,37.512561,127.058777,9.072810,31.818953,2019-09-30,2019-10-08,9,1,9,가을,0,48,2019,9,360.0,49040.0
2,2023,선사문화축제,596.0,466174.0,27,서울특별시,강동구,암사2동,37.559854,127.130832,14.135464,39.269869,2023-10-08,2023-10-09,2,1,10,가을,0,33,2023,10,13520.0,49390.0


#### 4) 시군구명, 행정동명, 연도 기준으로 df3 병합

In [388]:
df3.rename(columns={"연도":"개최연도"}, inplace=True)

In [389]:
df_final = pd.merge(df_final, df3, on=['시군구명', '행정동명', '개최연도'], how='left')

In [390]:
# 결측치 보간(평균)
columns_to_fill = ['10대_인구수', '20대_인구수', '30대_인구수', '40대_인구수', '50대_인구수', '60대이상_인구수', '평균 나이', '고령 지수']

## 1) 행정동명, 개최연도 기준 결측치 보간
mean_values = df3.groupby(['행정동명', '개최연도'])[columns_to_fill].mean()
for column in columns_to_fill:
    df_final[column] = df_final.apply(lambda row: mean_values.loc[(row['행정동명'], row['개최연도']), column]
                              if pd.isnull(row[column]) and (row['행정동명'], row['개최연도']) in mean_values.index 
                              else row[column], axis=1)

## 2) 행정동명 기준 결측치 보간
mean_values = df3.groupby(['행정동명'])[columns_to_fill].mean()
for column in columns_to_fill:
    df_final[column] = df_final.apply(lambda row: mean_values.loc[row['행정동명'], column]
                              if pd.isnull(row[column]) and row['행정동명'] in mean_values.index 
                              else row[column], axis=1)
    
## 3) 시군구명, 개최연도 기준 결측치 보간
mean_values = df3.groupby(['시군구명', '개최연도'])[columns_to_fill].mean()
for column in columns_to_fill:
    df_final[column] = df_final.apply(lambda row: mean_values.loc[(row['시군구명'], row['개최연도']), column]
                              if pd.isnull(row[column]) and (row['시군구명'], row['개최연도']) in mean_values.index 
                              else row[column], axis=1)
    
## 4) 시군구명 기준 결측치 보간
mean_values = df3.groupby(['시군구명'])[columns_to_fill].mean()
for column in columns_to_fill:
    df_final[column] = df_final.apply(lambda row: mean_values.loc[row['시군구명'], column]
                              if pd.isnull(row[column]) and row['시군구명'] in mean_values.index 
                              else row[column], axis=1)
    
## 5) 시도명, 개최연도 기준 결측치 보간
mean_values = df_final.groupby(['시도명', '개최연도'])[columns_to_fill].mean()
for column in columns_to_fill:
    df_final[column] = df_final.apply(lambda row: mean_values.loc[(row['시도명'], row['개최연도']), column]
                              if pd.isnull(row[column]) and (row['시도명'], row['개최연도']) in mean_values.index 
                              else row[column], axis=1)
    
## 6) 시도명 기준 결측치 보간
mean_values = df_final.groupby(['시도명'])[columns_to_fill].mean()
for column in columns_to_fill:
    df_final[column] = df_final.apply(lambda row: mean_values.loc[row['시도명'], column]
                              if pd.isnull(row[column]) and row['시도명'] in mean_values.index 
                              else row[column], axis=1)

#### 5) 시군구명 기준 df7 병합

In [391]:
df_final = pd.merge(df_final, df7, left_on='시군구명', right_on='시군구명_1', how='left')
df_final.head(3)

,개최연도,축제명,예산(백만원),방문객수,개최 기간,시도명,시군구명,행정동명,위도,경도,서울과의 거리,광역시와의 거리,축제시작일,축제종료일,축제 일수,주말/공휴일 유무,개최 월,개최 계절,KTX역 유무,동일 기간 축제 수,연도,월,검색량,검색량(3개월),10대_인구수,20대_인구수,30대_인구수,40대_인구수,50대_인구수,평균 나이,60대이상_인구수,고령 지수,시군구명_1,소득2천만원주민비율,소득3천만원주민비율,소득4천만원주민비율,소득5천만원주민비율,소득6천만원주민비율,소득7천만원주민비율,소득7천만원이상주민비율
0,2023,강남페스티벌,2532.0,56894.0,11,서울특별시,강남구,삼성1동,37.512561,127.058777,9.072810,31.818953,2023-09-30,2023-10-08,9,1,9,가을,0,49,2023,9,12640.0,47900.0,1123.00,653.0,1555.0,1715.00,1920.0,54.190947,5184.0,0.426667,강남구,0.08,0.28,0.21,0.13,0.07,0.05,0.18
1,2019,강남페스티벌,2233.0,312502000.0,7,서울특별시,강남구,삼성1동,37.512561,127.058777,9.072810,31.818953,2019-09-30,2019-10-08,9,1,9,가을,0,48,2019,9,360.0,49040.0,1414.00,975.0,2092.0,2196.00,2432.0,51.897128,5544.0,0.378198,강남구,0.08,0.28,0.21,0.13,0.07,0.05,0.18
2,2023,선사문화축제,596.0,466174.0,27,서울특별시,강동구,암사2동,37.559854,127.130832,14.135464,39.269869,2023-10-08,2023-10-09,2,1,10,가을,0,33,2023,10,13520.0,49390.0,1943.05,1544.6,2822.6,3430.75,3719.1,53.737367,9494.4,0.411603,강동구,0.09,0.34,0.26,0.13,0.06,0.04,0.08


#### 6) 축제 시작일 기준 df5 병합

In [392]:
# 축제 개최 지역과 가장 가까운 관측 지점을 매핑하는 함수
df5['일시'] = pd.to_datetime(df5['일시'])
def find_nearest(df_final, df5):
    min_dist = float('inf')
    nearest_idx = None

    for index, row in df5.iterrows():
        # 축제시작일의 날씨 데이터를 가져옴
        if df_final['축제시작일'] == row['일시']:  
            distance = haversine(df_final['위도'], df_final['경도'], row['위도'], row['경도'])
            if distance < min_dist:
                min_dist = distance
                nearest_idx = index

    return df5.loc[nearest_idx]

In [394]:
# 축제시작일의 가장 가까운 관측 지점에서의 날씨를 가져오는 함수
def merge_weather(df_final, df5):
    df = df_final.copy()

    for index, row in tqdm(df_final.iterrows()):
        nearest_row = find_nearest(row, df5)
        df.loc[index, '일시'] = nearest_row['일시']
        df.loc[index, '평균기온(°C)'] = nearest_row['평균기온(°C)']
        df.loc[index, '평균 풍속(m/s)'] = nearest_row['평균 풍속(m/s)']
        df.loc[index, '평균 상대습도(%)'] = nearest_row['평균 상대습도(%)']
        df.loc[index, '비 유무'] = nearest_row['비 유무']

    return df

final = merge_weather(df_final, df5)

856it [33:05,  2.32s/it]


In [395]:
final.head(1)

,개최연도,축제명,예산(백만원),방문객수,개최 기간,시도명,시군구명,행정동명,위도,경도,서울과의 거리,광역시와의 거리,축제시작일,축제종료일,축제 일수,주말/공휴일 유무,개최 월,개최 계절,KTX역 유무,동일 기간 축제 수,연도,월,검색량,검색량(3개월),10대_인구수,20대_인구수,30대_인구수,40대_인구수,50대_인구수,평균 나이,60대이상_인구수,고령 지수,시군구명_1,소득2천만원주민비율,소득3천만원주민비율,소득4천만원주민비율,소득5천만원주민비율,소득6천만원주민비율,소득7천만원주민비율,소득7천만원이상주민비율,일시,평균기온(°C),평균 풍속(m/s),평균 상대습도(%),비 유무
0,2023,강남페스티벌,2532.0,56894.0,11,서울특별시,강남구,삼성1동,37.512561,127.058777,9.07281,31.818953,2023-09-30,2023-10-08,9,1,9,가을,0,49,2023,9,12640.0,47900.0,1123.0,653.0,1555.0,1715.0,1920.0,54.190947,5184.0,0.426667,강남구,0.08,0.28,0.21,0.13,0.07,0.05,0.18,2023-09-30,19.7,2.2,77.0,1.0


In [397]:
final.drop(["연도", "월", "검색량", "시군구명_1", "일시"], axis=1, inplace=True)

In [398]:
final.head()

,개최연도,축제명,예산(백만원),방문객수,개최 기간,시도명,시군구명,행정동명,위도,경도,서울과의 거리,광역시와의 거리,축제시작일,축제종료일,축제 일수,주말/공휴일 유무,개최 월,개최 계절,KTX역 유무,동일 기간 축제 수,검색량(3개월),10대_인구수,20대_인구수,30대_인구수,40대_인구수,50대_인구수,평균 나이,60대이상_인구수,고령 지수,소득2천만원주민비율,소득3천만원주민비율,소득4천만원주민비율,소득5천만원주민비율,소득6천만원주민비율,소득7천만원주민비율,소득7천만원이상주민비율,평균기온(°C),평균 풍속(m/s),평균 상대습도(%),비 유무
0,2023,강남페스티벌,2532.0,56894.0,11,서울특별시,강남구,삼성1동,37.512561,127.058777,9.072810,31.818953,2023-09-30,2023-10-08,9,1,9,가을,0,49,47900.0,1123.00,653.00,1555.00,1715.00,1920.00,54.190947,5184.00,0.426667,0.08,0.28,0.21,0.13,0.07,0.05,0.18,19.7,2.2,77.0,1.0
1,2019,강남페스티벌,2233.0,312502000.0,7,서울특별시,강남구,삼성1동,37.512561,127.058777,9.072810,31.818953,2019-09-30,2019-10-08,9,1,9,가을,0,48,49040.0,1414.00,975.00,2092.00,2196.00,2432.00,51.897128,5544.00,0.378198,0.08,0.28,0.21,0.13,0.07,0.05,0.18,22.4,1.6,61.0,0.0
2,2023,선사문화축제,596.0,466174.0,27,서울특별시,강동구,암사2동,37.559854,127.130832,14.135464,39.269869,2023-10-08,2023-10-09,2,1,10,가을,0,33,49390.0,1943.05,1544.60,2822.60,3430.75,3719.10,53.737367,9494.40,0.411603,0.09,0.34,0.26,0.13,0.06,0.04,0.08,17.1,2.5,67.1,0.0
3,2021,선사문화축제,617.0,10000.0,25,서울특별시,강동구,암사2동,37.559854,127.130832,14.135464,39.269869,2021-10-08,2021-10-09,2,1,10,가을,0,12,13967.5,1942.50,1716.05,3023.35,3554.85,3742.70,53.060661,9149.75,0.399161,0.09,0.34,0.26,0.13,0.06,0.04,0.08,18.8,1.7,91.3,1.0
4,2018,선사문화축제,640.0,400000.0,22,서울특별시,강동구,암사2동,37.559854,127.130832,14.135464,39.269869,2018-10-08,2018-10-09,2,1,10,가을,0,33,64.0,1961.40,1589.55,2910.60,3361.65,3446.55,52.296036,8098.40,0.389324,0.09,0.34,0.26,0.13,0.06,0.04,0.08,14.8,1.1,47.0,0.0


In [400]:
# final.to_csv(path + "data_final.csv", index=False)